In [59]:
import numpy as np
import iris
from iris.coord_categorisation import add_year
from iris.coord_categorisation import add_month_number
from iris.coord_categorisation import add_day_of_month
import iris.quickplot as qplt
import iris.plot as iplt
import matplotlib.pyplot as plt
import cf_units as unit
from scipy.signal import find_peaks
%matplotlib

Using matplotlib backend: Qt5Agg


In [112]:
def merge_composites(comp, lags=50, runid='obs'):
    # Comp is a list of cubes
    # Merging composites
    time_unit = unit.Unit('days since 2001-01-01 00:00:00', calendar=unit.CALENDAR_GREGORIAN)
    time_coord = iris.coords.DimCoord(np.arange(-lag, lag + 1), long_name='time', units=time_unit)
    print(time_coord)
    cube_list = iris.cube.CubeList(comp)
    for n, cube in enumerate(cube_list):
        print(cube.shape)
        if cube.coords('realization'):
            cube.remove_coord('realization')
        else:
            cube.add_aux_coord(iris.coords.AuxCoord(n, long_name='realization'))
        cube.remove_coord('time')
        cube.add_dim_coord(time_coord, 0)
        if not runid == 'obs':
            cube.remove_coord('forecast_period')
            
    comp = cube_list.merge_cube()
    return comp

In [124]:
cube = iris.load_cube('/project/MJO_GCSS/hadgem3/data/obs/SEAPy_data/obs/obs_precipitation_flux_MJO_filtered.nc')
cube = cube[400:2000]
ntime, nlat, nlon = cube.shape

In [114]:
cube_ts = cube.intersection(latitude=(-10,10), longitude=(80, 100)).collapsed(['latitude', 'longitude'], iris.analysis.MEAN)

/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/cube.py:3180: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1355: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1355: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))


In [115]:
#Normalise
cube_ts_normal = (cube_ts - cube_ts.collapsed('time', iris.analysis.MEAN))/cube_ts.collapsed('time', iris.analysis.STD_DEV)
plt.plot(cube_ts_normal.data)

/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1355: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1355: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'day_of_month'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1355: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'month_number'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1355: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'year'.
  warnings.warn(msg.format(self.name()))


In [116]:
# Find peaks above 1 sd
peaks, _ = find_peaks(cube_ts_normal.data, height=1.)

In [117]:
peaks, cube_ts_normal.data[peaks]
plt.plot(cube_ts_normal.data)
plt.plot(peaks, cube_ts_normal.data[peaks], '*')

In [127]:
ntime, = cube_ts_normal.shape
npeaks = len(peaks)
ntime, npeaks

(1600, 24)

In [146]:
ntime, = cube_ts_normal.shape
lag = 50 # days to be extracted around the peak (total window is -lag to +lag)
comp = np.ndarray((npeaks, 2*lag+1, nlat, nlon), dtype=float)
comp_cube = cube[:2*lag+1]

comp.shape
for npk, peak in enumerate(peaks):
    if peak >= lag and peak <= ntime - lag-1:
        # Append the composite
        comp[npk] = cube[peak - lag:peak + lag + 1].data

In [149]:
# Composite of all events
comp_cube.data = np.mean(comp, axis=0)

qplt.pcolormesh(comp_cube[:,30,:])

/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1138: UserWarning: Coordinate 'longitude' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))
/opt/scitools/environments/default/2019_02_27/lib/python3.6/site-packages/iris/coords.py:1138: UserWarning: Coordinate 'time' is not bounded, guessing contiguous bounds.
  'contiguous bounds.'.format(self.name()))


In [139]:
comp_cube

Precipitation Flux (mm),time,latitude,longitude
Shape,102,61,360
Dimension coordinates,,,
time,x,-,-
latitude,-,x,-
longitude,-,-,x
Auxiliary coordinates,,,
day_of_month,x,-,-
month_number,x,-,-
year,x,-,-
Attributes,,,
